In [ ]:
!pip install transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 114.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 95.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

In [ ]:
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

In [ ]:
def load_model(model_name="SalesForce/codegen-350M-mono"):
    """
    Load the CodeLlama model and tokenizer, and create a text-generation pipeline.
    Adjust device_map if needed.
    """
    token = os.environ["HF_TOKEN"]
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")
    generator = pipeline("text-generation", model=model, torch_dtype=torch.float16, tokenizer=tokenizer)
    return generator

In [ ]:
def benchmark_error_detection(generator, verilog_samples, expected_errors, prompt_template):
    """
    For each Verilog code sample:
    - Format a prompt using the prompt_template.
    - Use the model to generate output.
    - Check if the expected error substring is in the generated text.
    Returns the proportion of samples for which the expected error was found.
    """
    correct = 0
    total = len(verilog_samples)

    for idx, (code, expected_error) in enumerate(zip(verilog_samples, expected_errors)):
        # Create prompt by injecting the Verilog code into the template
        prompt = prompt_template.format(code=code)
        print(f"--- Sample {idx+1} ---")
        print("Prompt:")
        print(prompt)

        # Generate model output
        result = generator(prompt, max_new_tokens=128, do_sample=False, return_full_text=False)
        generated_text = result[0]['generated_text']
        print("Model Output:")
        print(generated_text)

        # Check if expected error substring appears in the output
        if expected_error.lower() in generated_text.lower():
            print("-> Expected error detected.\n")
            correct += 1
        else:
            print("-> Expected error NOT detected.\n")

    accuracy = correct / total if total > 0 else 0
    return accuracy

In [ ]:
if __name__ == "__main__":
    # Define a list of Verilog code samples with errors.
        # Specify the CSV file path (CSV should have columns "code" and "error")
    csv_file = "/content/output.csv"  # Change this to your CSV file's path

    # Read the CSV file into a DataFrame
    df = pd.read_csv(csv_file)
    verilog_samples = df["code"].tolist()
    expected_errors = df["error_type"].tolist()

    # Define the prompt template. {code} will be replaced by the actual Verilog code.
    prompt_template = "Analyze the following Verilog code /n/n{code}/n and provide exactly one error type from the list below. Your response must be one word only, nothing else: missing_semicolon, mismatched_parentheses, missing_or_extra_endmodule, missing_or_extra_endcase_endfunction_endgenerate, mismatched_begin_end, using_reserved_keyword_as_identifier, missing_or_extra_colon_in_case_item, malformed_timescale_directive, unclosed_block_comment, missing_or_extra_quote_in_string_literals, missing_or_extra_backtick_in_preprocessor_directives, incomplete_or_extra_array_brackets, malformed_generate_statement, missing_or_extra_comma_in_port_or_sensitivity_list, unclosed_string_in_system_task, malformed_macro_definition, premature_termination, undefined_variable, operator_misuse, redundant_variable_declaration, incorrect_encoding, incorrect_data_type_assignment, port_mode_declaration_error, data_index_out_of_bounds, improper_use_of_keywords, no_error, other_error"

    # Load the model
    generator = load_model()

    # Benchmark error detection
    accuracy = benchmark_error_detection(generator, verilog_samples, expected_errors, prompt_template)
    print(f"Overall Accuracy of Error Detection: {accuracy*100:.2f}%")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/240 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

...

Model Output:
, missing_or_extra_endmodule, missing_or_extra_endfunction, missing_or_extra_endgenerate, missing_or_extra_endcase_endfunction_endgenerate, missing_or_extra_begin_end, missing_or_extra_begin_endmodule, missing_or_extra_begin_endcase_endfunction_endgenerate, missing_or_extra_begin_endmodule, missing_or_extra_begin_endcase_endfunction_endgenerate, missing_or_extra_begin_endmodule, missing_or_
-> Expected error NOT detected.

--- Sample 7 ---
Prompt:
Analyze the following Verilog code /n/n
module Ref)Module (
  input a,
  input b,
  output out
);

  assign out = a & b;

endmodule

/n and provide exactly one error type from the list below. Your response must be one word only, nothing else: missing_semicolon, mismatched_parentheses, missing_or_extra_endmodule, missing_or_extra_endcase_endfunction_endgenerate, mismatched_begin_end, using_reserved_keyword_as_identifier, missing_or_extra_colon_in_case_item, malformed_timescale_directive, unclosed_block_comment, missing_or_extra_q

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Model Output:
, missing_or_extra_endmodule, missing_or_extra_endfunction, missing_or_extra_endgenerate, missing_or_extra_endgenerate_endfunction, missing_or_extra_endgenerate_endfunction_endgenerate, missing_or_extra_endgenerate_endfunction_endgenerate, missing_or_extra_endgenerate_endfunction_endgenerate, missing_or_extra_endgenerate_endfunction_endgenerate, missing_or_extra_endgenerate_endfunction_endgenerate, missing
-> Expected error NOT detected.

--- Sample 8 ---
Prompt:
Analyze the following Verilog code /n/n
module Ref)Module (
  input a,
  input b,
  output q
);

  assign q = a&b;

endmodule

/n and provide exactly one error type from the list below. Your response must be one word only, nothing else: missing_semicolon, mismatched_parentheses, missing_or_extra_endmodule, missing_or_extra_endcase_endfunction_endgenerate, mismatched_begin_end, using_reserved_keyword_as_identifier, missing_or_extra_colon_in_case_item, malformed_timescale_directive, unclosed_block_comment, missing_

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Model Output:
, missing_or_extra_endmodule, missing_or_extra_endfunction, missing_or_extra_endgenerate, missing_or_extra_endgenerate_endfunction, missing_or_extra_endgenerate_endfunction_endgenerate, missing_or_extra_endgenerate_endfunction_endgenerate, missing_or_extra_endgenerate_endfunction_endgenerate, missing_or_extra_endgenerate_endfunction_endgenerate, missing_or_extra_endgenerate_endfunction_endgenerate, missing
-> Expected error NOT detected.

--- Sample 9 ---
Prompt:
Analyze the following Verilog code /n/n
module RefMo
endfunction // Extra inserted
dule (
  input in,
  output out
);

  assign out = in;

endmodule

/n and provide exactly one error type from the list below. Your response must be one word only, nothing else: missing_semicolon, mismatched_parentheses, missing_or_extra_endmodule, missing_or_extra_endcase_endfunction_endgenerate, mismatched_begin_end, using_reserved_keyword_as_identifier, missing_or_extra_colon_in_case_item, malformed_timescale_directive, unclosed_